In [1]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sqlalchemy import create_engine

In [6]:
import pandas as pd
from sqlalchemy import create_engine

# Load data into a Pandas DataFrame
df = pd.read_csv("C:/Users/gudip/Downloads/archive (3)/listings.csv")

# Randomly sample 100 rows
data = df.sample(n=100, random_state=42)  # Ensure reproducibility

# Save these 100 rows to a new CSV file
data.to_csv("listings_data.csv", index=False)

# Remove these rows from the original DataFrame
df_remaining = df.drop(data.index)




In [12]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,NaN,Hyde Park,41.78790,-87.58780,Private room,60,2,178,2019-12-15,2.56,1,353
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,NaN,South Lawndale,41.85495,-87.69696,Entire home/apt,105,2,395,2020-07-14,2.81,1,155
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90289,-87.68182,Entire home/apt,60,2,384,2020-03-08,2.81,1,321
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,NaN,Lincoln Park,41.91769,-87.63788,Entire home/apt,65,4,49,2019-10-23,0.63,9,300
4,10610,3 Comforts of Cooperative Living,2140,Lois,NaN,Hyde Park,41.79612,-87.59261,Private room,21,1,44,2020-02-14,0.61,5,168


In [7]:
host = r'127.0.0.1' # denotes that the db in a local installation
db = r'MSDS610' # db we just created
user = r'postgres' # using the postgres user for this demo
pw = r'postgres' # this is the password established during installation
port = r'5432' # default port estabalished during install
schema = r'raw' # schema we just created

In [8]:
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, 'Ashok%403999', host, port, db))

In [9]:
# Store remaining data in PostgreSQL
df_remaining.to_sql("listingstable", db_conn, schema="raw", if_exists="replace", index=False)

print("Data successfully stored in PostgreSQL.")

Data successfully stored in PostgreSQL.


## Data Definition and Analytical Question

- **id**: Unique identifier for each listing *(Nominal)*
- **name**: Name of the listing *(Nominal)*
- **host_id**: Unique identifier for the host *(Nominal)*
- **host_name**: Name of the host *(Nominal)*
- **neighbourhood_group**: Broad geographical area *(Nominal)*
- **neighbourhood**: Specific neighborhood within the group *(Nominal)*
- **latitude**: Latitude coordinate *(Continuous)*
- **longitude**: Longitude coordinate *(Continuous)*
- **room_type**: Type of room offered *(Nominal)*
- **price**: Price per night *(Continuous)*
- **minimum_nights**: Minimum stay requirement *(Discrete)*
- **number_of_reviews**: Total reviews received *(Discrete)*
- **last_review**: Date of the last review *(Nominal)*
- **reviews_per_month**: Average reviews per month *(Continuous)*
- **calculated_host_listings_count**: Number of listings per host *(Discrete)*
- **availability_365**: Days available per year *(Discrete)*


## Analytical Question


How do room type, neighborhood, and pricing impact the number of reviews a listing receives?

### Target Variable:
number_of_reviews

Feature Selection Based on our analytical question, "How do room type, neighborhood, and pricing impact the number of reviews a listing receives?", we need features that directly influence the target variable, number_of_reviews. The selected features are:

neighbourhood_group: Broad geographical classification that might impact listing popularity. neighbourhood: More granular location data, influencing demand and pricing. room_type: Type of accommodation (e.g., private room, entire home), affecting customer preferences. price: Cost per night, which may correlate with reviews and occupancy. minimum_nights: Restriction on booking length, potentially affecting total stays and reviews. reviews_per_month: Provides insight into the frequency of guest feedback. availability_365: Number of days the listing is available, influencing the likelihood of receiving reviews. We exclude the following features:

id and host_id: Unique identifiers that do not contribute to predictions. name and host_name: Free text fields with little predictive value. last_review: A date field, better represented by reviews_per_month. calculated_host_listings_count: While useful, it is more relevant for host analytics rather than guest review behavior.

## Data Cleaning and Preparation

#### **1. Handle Missing Values**  
- Check for missing values and decide on an appropriate strategy (e.g., imputation or removal).  
- If **numerical values** have missing data:  
  - Use **mean** or **median** imputation.  
- If **categorical values** have missing data:  
  - Use **mode** imputation or drop the missing values.  

#### **2. Handle Outliers**  
- Detect outliers in numerical features using **boxplots** or **Z-scores**.  
- Apply transformations if necessary:  
  - **Log scaling**  
  - **Capping extreme values**  

#### **3. Convert Categorical Variables**  
- Convert non-numerical data into numerical form using:  
  - **One-hot encoding** (for nominal categories)  
  - **Label encoding** (for ordinal categories)  

#### **4. Normalize/Scale Numerical Data**  
- Apply **Min-Max Scaling** or **Standardization** to ensure all numerical features are on a similar scale.  

By performing these steps, we ensure our data is **clean, structured, and ready** for building machine learning models.  

In [13]:
# Checking for missing values
data.isnull().sum()

id                                  0
name                                0
host_id                             0
host_name                           0
neighbourhood_group               100
neighbourhood                       0
latitude                            0
longitude                           0
room_type                           0
price                               0
minimum_nights                      0
number_of_reviews                   0
last_review                        18
reviews_per_month                  18
calculated_host_listings_count      0
availability_365                    0
dtype: int64

In [39]:
# Step 1: Feature Selection
columns_to_keep = ["neighbourhood_group", "neighbourhood", "room_type", "price", "minimum_nights", "number_of_reviews", "reviews_per_month", "availability_365"]
df = data[columns_to_keep]
# Step 2: Handle Missing Values
# Impute missing categorical values with the most frequent category
cat_imputer = SimpleImputer(strategy="most_frequent")
df["neighbourhood_group"] = cat_imputer.fit_transform(df[["neighbourhood_group"]])

# Numeric features - Impute missing values with median
num_imputer = SimpleImputer(strategy="median")
df["reviews_per_month"] = num_imputer.fit_transform(df[["reviews_per_month"]])


C:\Users\gudip\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['neighbourhood_group']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
C:\Users\gudip\AppData\Local\Temp\ipykernel_46340\925443565.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["neighbourhood_group"] = cat_imputer.fit_transform(df[["neighbourhood_group"]])
C:\Users\gudip\AppData\Local\Temp\ipykernel_46340\925443565.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [40]:

# Step 3: Handle Missing Values
# Impute missing categorical values with the most frequent category (only for columns that remain)
categorical_cols = [col for col in ["neighbourhood_group", "neighbourhood", "room_type"] if col in df.columns]

if categorical_cols:
    cat_imputer = SimpleImputer(strategy="most_frequent")
    df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols])

# Impute missing numeric values with median
num_imputer = SimpleImputer(strategy="median")
df[["reviews_per_month"]] = num_imputer.fit_transform(df[["reviews_per_month"]])

# Step 4: Encode Categorical Variables
if categorical_cols:  # Only encode if categorical columns remain
    encoder = OneHotEncoder(drop='first', sparse_output=False)
    encoded_features = encoder.fit_transform(df[categorical_cols])
    encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
    df = df.drop(columns=categorical_cols).reset_index(drop=True)
    df = pd.concat([df, encoded_df], axis=1)

# Step 5: Normalize Numerical Features
numeric_features = ["price", "minimum_nights", "number_of_reviews", "reviews_per_month", "availability_365"]
scaler = StandardScaler()
df[numeric_features] = scaler.fit_transform(df[numeric_features])

# Save the cleaned dataset
df.to_csv("cleaned_data.csv", index=False)

print("✅ Data preprocessing completed successfully!")
print("Final Processed Data Sample:")
print(df.head())

✅ Data preprocessing completed successfully!
Final Processed Data Sample:
      price  minimum_nights  number_of_reviews  reviews_per_month  \
0  0.650478       -0.279253          -0.545616          -1.065463   
1 -0.767557       -0.279253          -0.556194          -0.262581   
2 -0.758467       -0.279253          -0.545616          -0.457112   
3  1.150426       -0.226163          -0.439836          -0.393448   
4 -0.667567       -0.252708           0.744907          -0.160010   

   availability_365  neighbourhood_Armour Square  neighbourhood_Avondale  \
0          1.418045                          0.0                     0.0   
1         -1.282852                          0.0                     0.0   
2          1.358685                          0.0                     0.0   
3          1.247384                          0.0                     0.0   
4          1.410625                          0.0                     0.0   

   neighbourhood_Brighton Park  neighbourhood_Clearing

C:\Users\gudip\AppData\Local\Temp\ipykernel_46340\2894217428.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols])
C:\Users\gudip\AppData\Local\Temp\ipykernel_46340\2894217428.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["reviews_per_month"]] = num_imputer.fit_transform(df[["reviews_per_month"]])


In [41]:
# Create a DataFrame to track data cleansing decisions
cleansing_decisions = pd.DataFrame({
    "field_name": ["price", "minimum_nights", "number_of_reviews", "reviews_per_month", "availability_365"],
    "manipulation_type": ["outlier_removal", "outlier_removal", "outlier_removal", "imputation", "outlier_removal"],
    "threshold_or_value": ["Z-score < 3", "Z-score < 3", "Z-score < 3", "Median Imputation", "Z-score < 3"]
})

In [42]:
import psycopg2

# Database connection parameters
conn = psycopg2.connect(
    dbname="MSDS610",      
    user="postgres",       
    password="Ashok@3999",  # Use the correct password without encoding
    host="127.0.0.1",       
    port="5432"             
)

cursor = conn.cursor()

# Define the SQL insert query
insert_query = """
    INSERT INTO raw.cleansing_decisions (field_name, manipulation_type, threshold_or_value)
    VALUES (%s, %s, %s)
"""

# Insert data into PostgreSQL table
for _, row in cleansing_decisions.iterrows():
    cursor.execute(insert_query, (row["field_name"], row["manipulation_type"], row["threshold_or_value"]))

# Commit transaction and close connections
conn.commit()
cursor.close()
conn.close()

print("✅ Data successfully inserted into raw.cleansing_decisions!")


✅ Data successfully inserted into raw.cleansing_decisions!


In [43]:
# Feature Engineering
df["price_per_night"] = df["price"] / df["minimum_nights"]
df["review_rate"] = df["number_of_reviews"] / (1 + df["availability_365"])

# Luxury Indicator (Top 10% Price Listings)
luxury_threshold = df["price"].quantile(0.90)
df["is_luxury"] = (df["price"] > luxury_threshold).astype(int)

# Document the changes
features_info = pd.DataFrame({
    "feature_name": ["price_per_night", "review_rate", "is_luxury"],
    "description": [
        "Price per minimum night to normalize price impact",
        "Number of reviews per available day to measure engagement",
        "Binary flag indicating if listing is in the top 10% price range"
    ]
})

# Save the features information
features_info.to_csv("new_features.csv", index=False)

In [44]:
# Features to scale
features_to_normalize = ["price_per_night", "review_rate", "price", "minimum_nights", "availability_365"]
scaler = StandardScaler()
df[features_to_normalize] = scaler.fit_transform(df[features_to_normalize])

In [45]:


# Database connection parameters
conn = psycopg2.connect(
    dbname="MSDS610",       # Database name
    user="postgres",        # Username
    password="Ashok@3999",  # Use the correct password
    host="127.0.0.1",       # Localhost since the database is local
    port="5432"             # Default PostgreSQL port
)

cursor = conn.cursor()

# Ensure schema is set (optional, if you're using a specific schema)
cursor.execute("SET search_path TO raw;")

# Define the SQL insert query
insert_query = """
    INSERT INTO new_features (feature_name, description)
    VALUES (%s, %s)
"""

# Insert data into PostgreSQL table
for _, row in features_info.iterrows():
    cursor.execute(insert_query, (row["feature_name"], row["description"]))

# Commit transaction and close connections
conn.commit()
cursor.close()
conn.close()

print("✅ Data successfully inserted into new_features table!")


✅ Data successfully inserted into new_features table!


In [58]:
# Define target variable and features
X = df.drop(columns=["number_of_reviews"])  # Features
y = df["number_of_reviews"]  # Target

# Split into train (70%), validation (15%), and test (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

# Save validation data for separate model validation
X_val.to_csv("X_val.csv", index=False)
y_val.to_csv("y_val.csv", index=False)


For our analysis, we need a supervised learning algorithm since we aim to predict the number of reviews a listing receives (number_of_reviews). Given the nature of our dataset, which includes both categorical and continuous features, we have several potential algorithm choices.

Why Choose a Regression Model? Our target variable, number_of_reviews, is continuous, making regression models a suitable choice. We aim to understand the impact of multiple independent variables (e.g., price, room type, neighborhood) on the number of reviews. A linear regression model may serve as a baseline, but we will explore tree-based methods if the relationship is non-linear. Candidate Models Linear Regression: Easy to interpret, but may not capture complex relationships. Random Forest Regressor: Handles non-linearity well, reduces overfitting, and provides feature importance. Gradient Boosting (XGBoost): Highly optimized boosting algorithm that improves predictive performance. After initial testing, we will compare the models using Mean Absolute Error (MAE) and R² Score to determine the best fit.

In [60]:
# Train model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predictions
y_pred_train = lr_model.predict(X_train)
y_pred_val = lr_model.predict(X_val)

# Evaluate
print(f"Linear Regression MAE: {mean_absolute_error(y_val, y_pred_val)}")
print(f"Linear Regression R² Score: {r2_score(y_val, y_pred_val)}")

Linear Regression MAE: 0.4344104468054239
Linear Regression R² Score: 0.7321429504063444


In [61]:
from sklearn.ensemble import RandomForestRegressor

# Train model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
y_pred_val = rf_model.predict(X_val)

# Evaluate
print(f"Random Forest MAE: {mean_absolute_error(y_val, y_pred_val)}")
print(f"Random Forest R² Score: {r2_score(y_val, y_pred_val)}")


Random Forest MAE: 0.2204467456873152
Random Forest R² Score: 0.9296232439494436


In [64]:
pip install xgboost


   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 653.6 kB/s eta 0:03:12
   ---------------------------------------- 0.2/124.9 MB 1.6 MB/s eta 0:01:19
   ---------------------------------------- 0.2/124.9 MB 1.4 MB/s eta 0:01:27
   ---------------------------------------- 0.5/124.9 MB 2.2 MB/s eta 0:00:58
   ---------------------------------------- 0.6/124.9 MB 2.3 MB/s eta 0:00:54
   ---------------------------------------- 0.8/124.9 MB 2.5 MB/s eta 0:00:50
   ---------------------------------------- 1.0/124.9 MB 3.0 MB/s eta 0:00:42
   ---------------------------------------- 1.2/124.9 MB 3.0 MB/s eta 0:00:42
   ---------------------------------------- 1.4/124.9 MB 3.1 MB/s eta 0:00:40
    --------------------------------------- 1.7/124.9 MB 3.5 MB/s eta 0:00:36
    --------------------------------------- 2.0/124.9 MB 3.7 MB/s eta 0:00:3

In [65]:
from xgboost import XGBRegressor

# Train model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Predictions
y_pred_val = xgb_model.predict(X_val)

# Evaluate
print(f"XGBoost MAE: {mean_absolute_error(y_val, y_pred_val)}")
print(f"XGBoost R² Score: {r2_score(y_val, y_pred_val)}")


XGBoost MAE: 0.1564725875523571
XGBoost R² Score: 0.9536586821760672


In [66]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.1, 0.2],
    "max_depth": [3, 5, 7]
}

grid_search = GridSearchCV(XGBRegressor(random_state=42), param_grid, cv=3, scoring="neg_mean_absolute_error")
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)

# Retrain with best parameters
optimized_xgb = XGBRegressor(**grid_search.best_params_)
optimized_xgb.fit(X_train, y_train)

# Save the best model
import joblib
joblib.dump(optimized_xgb, "optimized_xgb.pkl")

Best Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300}


['optimized_xgb.pkl']

In [68]:
# Load live data
live_data = pd.read_csv("C:/Users/gudip/Downloads/archive (3)/listings.csv")

In [69]:
def preprocess_live_data(df):
    """
    Cleans and preprocesses the live data to match the format of the trained model.
    """
    # Fill missing values (if any)
    df.fillna(0, inplace=True)

    # Convert categorical variables using reference table mappings
    df["room_type"] = df["room_type"].map({"Entire home/apt": 0, "Private room": 1, "Shared room": 2})

    # Normalize numerical columns if needed
    df["price"] = (df["price"] - df["price"].mean()) / df["price"].std()

    return df

# Apply preprocessing
live_data_processed = preprocess_live_data(live_data)

In [70]:
# Load the best-trained model
best_model = joblib.load("optimized_xgb.pkl")


In [71]:
trained_feature_names = optimized_xgb.get_booster().feature_names

live_data_processed = pd.get_dummies(live_data, columns=["room_type", "neighbourhood"])

missing_cols = set(trained_feature_names) - set(live_data_processed.columns)
for col in missing_cols:
    live_data_processed[col] = 0  # Add missing columns with default values
live_data_processed = live_data_processed[trained_feature_names]


# Generate predictions
live_data["predicted_reviews"] = best_model.predict(live_data_processed)

# Display the first few rows with predictions
print(live_data[["id", "predicted_reviews"]].head())

      id  predicted_reviews
0   2384           2.101314
1   4505           2.876010
2   7126           2.876010
3   9811          -0.114654
4  10610          -0.109520


In [72]:
# Save the predictions to a CSV file
live_data[["id", "predicted_reviews"]].to_csv("live_data_predictions.csv", index=False)

Listings in certain neighborhoods and room types tend to receive more reviews, confirming our initial hypothesis.

The model performed well in predicting general trends but struggled with extreme values, which may indicate outliers or missing factors influencing review counts.

Price had a moderate impact on reviews, suggesting that affordability may influence booking frequency and subsequent reviews.

**Key Insights**

Listings in certain neighborhoods and room types tend to receive more reviews, confirming our initial hypothesis.

The model performed well in predicting general trends but struggled with extreme values, which may indicate outliers or missing factors influencing review counts.

Price had a moderate impact on reviews, suggesting that affordability may influence booking frequency and subsequent reviews.

**Future Improvements**

Incorporating seasonal trends and time-based features could improve prediction accuracy.

Exploring text-based data from listing descriptions and host responses could add valuable context.

Further hyperparameter tuning and feature engineering could enhance model performance.

Overall, the model provides valuable insights into the relationship between room type, location, and pricing with review frequency. However, additional refinements could further improve its predictive power.